In [2]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
import math
import numpy as np
from IPython.display import display

In [3]:
df_20150817 = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/morte_motoqueiro_20150817/20150817.txt', delimiter = ('\t'))

In [4]:
df_20150817_line_316 = df_20150817[(df_20150817['line']=='316')]


In [7]:
df_20150817_line_316_parados = df_20150817_line_316[(df_20150817_line_316['speed'] == 0)]


In [5]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [6]:
onibus316_shapes = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Shapes/gtfs_linha316-shapes.csv')
fmap316=folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316 = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

fmap316.line(locations316)
embed_map(fmap316)

In [7]:
#esto es para graficar la porcion de la ruta que será analizada con las paradas representativas
for row in df_20150817_line_316_parados.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap316.polygon_marker( latlon, fill_color='red', num_sides=4, radius=6)
    
    

embed_map(fmap316)

In [8]:
#esto es para graficar la porcion de la ruta que será analizada con las paradas representativas
for row in df_20150817_line_316.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap316.polygon_marker( latlon, fill_color='green', num_sides=4, radius=6)
    

embed_map(fmap316)

In [9]:
df_initial_point = onibus316_shapes[(onibus316_shapes['sequencia']==0)]
df_initial_point

,linha,descricao,agencia,sequencia,shape_id,latitude,longitude
0,316,316-RECREIO DOS BANDEIRANTES X CENTRAL (VIA BO...,Fetranspor,0,17342012,-23.0224,-43.4823
1274,316,316-RECREIO DOS BANDEIRANTES X CENTRAL (VIA BO...,Fetranspor,0,17418997,-22.9049,-43.1917


In [10]:
#Punto inicial del omnibus 316 Central do Brasil
initial_point = (-23.0224,-43.4823)
#Punto inicial del omnibus invertido
inverted_initial_point = (-43.4823, -23.0224)
#Punto final del omnibus 316 Centro
final_point = (-22.9049, -43.1917)
#Punto final del omnibus invertido
inverted_final_point = (-43.1917, -22.9049)


In [11]:
#Pintando Circulos de radio de 50m alrededor de los puntos incial y final 
fmap316.simple_marker(final_point, popup='Centro')
fmap316.simple_marker(initial_point, popup='Barra')

#Punto inicial
fmap316.circle_marker(location=final_point, radius=50,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
fmap316.circle_marker(location=initial_point, radius=50,line_color='blue',
                 fill_color='#3186cc', fill_opacity=0.2)

embed_map(fmap316)

In [12]:
#Estableciendo la conexion con la base de datos morte_motoquiro en MongoDB
db = MongoClient().morte_motoquiro_event

In [13]:
#Function to compute the bus_ids circulating during a day 20150208 of bus line 438
def bus_id_list ():
    bus_id_20150817_316 = db.dados20150817_line_316.distinct("bus_id")
    return bus_id_20150817_316

In [14]:
#llamada a la funcion bus_id_list
busId_list = bus_id_list()
busId_list

[u'C41422', u'C41424', u'C41427', u'C41423', u'C41429', u'C41426', u'C41421']

In [15]:
#Funcion para meter los bus_Id del dataframe para un array
def get_busId_list():
    array_busId = []
    for bus in busId_list:
        strbus = str(bus)
        array_busId.insert(len(array_busId), strbus)
    return array_busId

In [16]:
array_busId = get_busId_list()
array_busId

['C41422', 'C41424', 'C41427', 'C41423', 'C41429', 'C41426', 'C41421']

In [17]:
#funcion que dado un el bus_id de un omnibus y un punto inicial/final devuelve los puntos mas cercanos a ese punto. 
# A esa función se le debería pasar también la BD, pero ahora no se va a hacer asi
def near_points_to_origin (bus_id, inverted_origin_point):
    near_initial_points_array = []
    #la funcion $within recibe las coordenadas en orden inverso (primero longitud y después latitud)
    for row in db.dados20150817_line_316.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.0015499543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        near_initial_points_array.insert(0, [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'speed']])
    #convert the array in panda dataframe 
    sorted_near_initial_points = pd.DataFrame(near_initial_points_array, columns=['timestamp', 'latitude', 'longitude', 'speed'])
    return sorted_near_initial_points

In [18]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos a la barra 

sorted_near_initial_points = near_points_to_origin ('C41422', inverted_initial_point)


In [19]:
sorted_near_final_points = near_points_to_origin ('C41422', inverted_final_point)


In [20]:
test_map = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316_test = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316_test.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

test_map.line(locations316_test)

#insertar los popups
#Pintando Circulos de radio de 155m alrededor de los puntos incial y final 
test_map.simple_marker(final_point, popup='Centro')
test_map.simple_marker(initial_point, popup='Barra')

#Punto inicial
test_map.circle_marker(location=final_point, radius=155,line_color='brown',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
test_map.circle_marker(location=initial_point, radius=155,line_color='black',
                 fill_color='#3186cc', fill_opacity=0.2)

df_20150817_line_316[(df_20150817_line_316['bus_id'] == 'C41422')]

embed_map(test_map)

In [21]:
df_20150817_line_316_C41422 = df_20150817_line_316[(df_20150817_line_316['bus_id'] == 'C41422')]


In [22]:
len(df_20150817_line_316_C41422)

667

In [23]:
#esto es para graficar la porcion de la ruta que será analizada con las paradas representativas
for row in sorted_near_initial_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    test_map.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)
    
    

embed_map(test_map)

In [23]:
#esto es para graficar la porcion de la ruta que será analizada con las paradas representativas
for row in sorted_near_final_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    test_map.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)
    

embed_map(test_map)

In [24]:
#Array de timestamp puntos near punto incial (Leblon)
initial_points_timestamp = sorted_near_initial_points['timestamp']


In [25]:
final_points_timestamp = sorted_near_final_points['timestamp']


In [24]:
def bus_departure_hour (initial_points_timestamp_list):
    list_departure_hours = []
    i = 0
    if len(initial_points_timestamp_list) >1:
        for i in range(0, len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            #si la diferencia entre los timestamp del omnibus en el putno supera los 40 minutos es q dio un viaje y regreso, sino lo supera es que se mantuvo ese tiempo esperando en el punto
            if (resto_total > 40):
                 list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S"))
        #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de  list_departure_hours 
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif len(initial_points_timestamp_list) == 1:
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_departure_hours = []
    return list_departure_hours

In [27]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
initial_departure_hours = bus_departure_hour(initial_points_timestamp)
initial_departure_hours

[datetime.datetime(2015, 8, 17, 8, 9, 9),
 datetime.datetime(2015, 8, 17, 13, 52, 1)]

In [28]:
#Llamada a la funcion bus_departure_hour para el punto Vila Isabel  (devuelve todas las horas en que salio el omnibus de Vila Isabel)
final_departure_hours = bus_departure_hour(final_points_timestamp)
final_departure_hours

[datetime.datetime(2015, 8, 17, 5, 58, 58),
 datetime.datetime(2015, 8, 17, 11, 18, 58),
 datetime.datetime(2015, 8, 17, 17, 19, 48)]

In [46]:
#Funcion que retorna el arreglo con las horas de llegada del omnibus al punto inicial/final del trayecto
def bus_arrive_hour (initial_points_timestamp_list):
    list_arrive_hours = []
    #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de leblon_departure_hours 
    if len(initial_points_timestamp_list) >1:
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
        for i in range(0,  len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            if (resto_total > 40):
                list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif (len(initial_points_timestamp_list) == 1):
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_arrive_hours = []        
    return list_arrive_hours

In [30]:
#Llamada a la funcion bus_arrive_hour para el punto Leblon  (devuelve todas las horas en que llegó el omnibus a Leblon)
arrive_hour_to_initialpoint = bus_arrive_hour(initial_points_timestamp)
arrive_hour_to_initialpoint

[datetime.datetime(2015, 8, 17, 7, 59, 48),
 datetime.datetime(2015, 8, 17, 13, 47, 21)]

In [31]:
#Llamada a la funcion bus_arrive_hour para el punto Vila Isabel  (devuelve todas las horas en que llegó el omnibus a Vila Isabel)
arrive_hour_to_finalpoint = bus_arrive_hour(final_points_timestamp)
arrive_hour_to_finalpoint

[datetime.datetime(2015, 8, 17, 5, 54, 18),
 datetime.datetime(2015, 8, 17, 11, 7, 18),
 datetime.datetime(2015, 8, 17, 15, 50, 19)]

In [26]:
#Para convertir de formato timedelta para horas y minutos
def days_hours_minutes(travel_time):
    hours = travel_time.seconds//3600 
    minutes = (travel_time.seconds//60)%60
    #print '%d hours, %d minutes' % (hours,minutes)
    if minutes < 10:
        minutes = str(0)+str(minutes)
    return str(hours) + ":"+ str(minutes)

In [27]:
#Funcion que crea la tabla de viajes con los tiempos de viaje
def create_travel_table (name_A, arrive_array_A, departure_array_A, name_B, arrive_array_B, departure_array_B, bus_Id, bus_line):
    
    my_time_table = []   
    num_travel = 0    
    i = 0 #Cursor para moverme por el departure_array_A(arreglo q contiene las horas de salida del punto incial)   
    j = 0 #Cursor para moverme por el departure_array_B (arreglo q contiene las horas de salida del punto final)
   
    while (i<len(departure_array_A) and j< len(departure_array_B)):
        if (departure_array_A[i] < departure_array_B[j]):
            if(i+1 == len(departure_array_A)):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
         
            elif ((i+1 < len(departure_array_A)) and (departure_array_B[j] < departure_array_A[i+1])):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else: 
                i = i +1
            
        elif(departure_array_B[j] < departure_array_A[i]):
            if (j+1 == len(departure_array_B)):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            
            elif ((j+1 < len(departure_array_B)) and (departure_array_A[i] < departure_array_B[j+1])):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else:
                j = j +1                
            
    #Construir el dataframe pasandole el array con todos los datos de los viajes   
    mydf = pd.DataFrame(my_time_table, columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])       
    return mydf  


In [34]:
#dados para llamar a la funcion

mytabla_C41422 = create_travel_table("Barra", arrive_hour_to_initialpoint, initial_departure_hours, "Central", arrive_hour_to_finalpoint, final_departure_hours, 'C41422', '316' )
mytabla_C41422

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,316,C41422,Central_to_Barra,2015-08-17 05:58:58,2015-08-17 07:59:48,2:00
1,316,C41422,Barra_to_Central,2015-08-17 08:09:09,2015-08-17 11:07:18,2:58
2,316,C41422,Central_to_Barra,2015-08-17 11:18:58,2015-08-17 13:47:21,2:28
3,316,C41422,Barra_to_Central,2015-08-17 13:52:01,2015-08-17 15:50:19,1:58


In [35]:
#hacer lo mismo para el omnibus 'C41424'
df_20150817_line_316_C41424 = df_20150817_line_316[(df_20150817_line_316['bus_id'] == 'C41424')]

In [36]:
sorted_near_initial_points_C41424 = near_points_to_origin ('C41424', inverted_initial_point)

In [37]:
sorted_near_final_points_C41424 = near_points_to_origin ('C41424', inverted_final_point)


In [38]:
#graficando los initial y final points
map_316_C41424 = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316_C41424 = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316_C41424.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

map_316_C41424.line(locations316_C41424)

#insertar los popups
#Pintando Circulos de radio de 155m alrededor de los puntos incial y final 
map_316_C41424.simple_marker(final_point, popup='Centro')
map_316_C41424.simple_marker(initial_point, popup='Barra')

#Punto inicial
map_316_C41424.circle_marker(location=final_point, radius=155,line_color='brown',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
map_316_C41424.circle_marker(location=initial_point, radius=155,line_color='black',
                 fill_color='#3186cc', fill_opacity=0.2)

#esto es para graficar los near points de la parada inicial y final
for row in df_20150817_line_316_C41424.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41424.polygon_marker( latlon, fill_color='white', num_sides=4, radius=6)    

for row in sorted_near_initial_points_C41424 .iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41424.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)
    
for row in sorted_near_final_points_C41424.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41424.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)    

 

embed_map(map_316_C41424)


In [39]:
initial_points_timestamp_C41424 = sorted_near_initial_points_C41424['timestamp']


In [40]:
final_points_timestamp_C41424 = sorted_near_final_points_C41424['timestamp']

In [41]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
initial_departure_hours_C41424 = bus_departure_hour(initial_points_timestamp_C41424)
initial_departure_hours_C41424

[datetime.datetime(2015, 8, 17, 7, 34, 47),
 datetime.datetime(2015, 8, 17, 13, 9, 55),
 datetime.datetime(2015, 8, 17, 19, 9, 49)]

In [42]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
final_departure_hours_C41424 = bus_departure_hour(final_points_timestamp_C41424)
final_departure_hours_C41424

[datetime.datetime(2015, 8, 17, 10, 46, 10),
 datetime.datetime(2015, 8, 17, 15, 32, 6),
 datetime.datetime(2015, 8, 17, 21, 14, 21)]

In [43]:
arrive_hour_to_initialpoint_C41424 = bus_arrive_hour(initial_points_timestamp_C41424)
arrive_hour_to_initialpoint_C41424

[datetime.datetime(2015, 8, 17, 7, 13, 13),
 datetime.datetime(2015, 8, 17, 12, 59, 59),
 datetime.datetime(2015, 8, 17, 18, 53, 1)]

In [44]:
arrive_hour_to_finalpoint_C41424 = bus_arrive_hour(final_points_timestamp_C41424)
arrive_hour_to_finalpoint_C41424

[datetime.datetime(2015, 8, 17, 10, 16, 24),
 datetime.datetime(2015, 8, 17, 15, 14, 46),
 datetime.datetime(2015, 8, 17, 20, 59, 46)]

In [45]:
mytabla_C41424 = create_travel_table("Barra", arrive_hour_to_initialpoint_C41424, initial_departure_hours_C41424, "Central", arrive_hour_to_finalpoint_C41424, final_departure_hours_C41424, 'C41424', '316' )
mytabla_C41424

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,316,C41424,Barra_to_Central,2015-08-17 07:34:47,2015-08-17 10:16:24,2:41
1,316,C41424,Central_to_Barra,2015-08-17 10:46:10,2015-08-17 12:59:59,2:13
2,316,C41424,Barra_to_Central,2015-08-17 13:09:55,2015-08-17 15:14:46,2:04
3,316,C41424,Central_to_Barra,2015-08-17 15:32:06,2015-08-17 18:53:01,3:20
4,316,C41424,Barra_to_Central,2015-08-17 19:09:49,2015-08-17 20:59:46,1:49


In [51]:
#hacer lo mismo para el omnibus 'C41427'
df_20150817_line_316_C41427 = df_20150817_line_316[(df_20150817_line_316['bus_id'] == 'C41427')]

In [52]:
sorted_near_initial_points_C41427 = near_points_to_origin ('C41427', inverted_initial_point)


In [53]:
sorted_near_final_points_C41427 = near_points_to_origin ('C41427', inverted_final_point)


In [54]:
#graficando los initial y final points
map_316_C41427 = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316 = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

map_316_C41427.line(locations316)

#insertar los popups
#Pintando Circulos de radio de 155m alrededor de los puntos incial y final 
map_316_C41427.simple_marker(final_point, popup='Centro')
map_316_C41427.simple_marker(initial_point, popup='Barra')

#Punto inicial
map_316_C41427.circle_marker(location=final_point, radius=155,line_color='brown',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
map_316_C41427.circle_marker(location=initial_point, radius=155,line_color='black',
                 fill_color='#3186cc', fill_opacity=0.2)

#esto es para graficar los near points de la parada inicial y final
for row in df_20150817_line_316_C41427.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41427.polygon_marker( latlon, fill_color='white', num_sides=4, radius=6)    

for row in sorted_near_initial_points_C41427.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41427.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)
    
for row in sorted_near_final_points_C41427.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41427.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)    

embed_map(map_316_C41427)


In [55]:
initial_points_timestamp_C41427 = sorted_near_initial_points_C41427['timestamp']


In [56]:
final_points_timestamp_C41427 = sorted_near_final_points_C41427['timestamp']

In [57]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
initial_departure_hours_C41427 = bus_departure_hour(initial_points_timestamp_C41427)
initial_departure_hours_C41427

[datetime.datetime(2015, 8, 17, 9, 48, 50),
 datetime.datetime(2015, 8, 17, 17, 19, 21)]

In [58]:
final_departure_hours_C41427 = bus_departure_hour(final_points_timestamp_C41427)
final_departure_hours_C41427

[datetime.datetime(2015, 8, 17, 6, 36, 54),
 datetime.datetime(2015, 8, 17, 12, 15, 10),
 datetime.datetime(2015, 8, 17, 14, 41, 6),
 datetime.datetime(2015, 8, 17, 20, 39, 59)]

In [59]:
arrive_hour_to_initialpoint_C41427 = bus_arrive_hour(initial_points_timestamp_C41427)
arrive_hour_to_initialpoint_C41427

[datetime.datetime(2015, 8, 17, 9, 30, 57),
 datetime.datetime(2015, 8, 17, 17, 8, 51)]

In [60]:
arrive_hour_to_finalpoint_C41427 = bus_arrive_hour(final_points_timestamp_C41427)
arrive_hour_to_finalpoint_C41427

[datetime.datetime(2015, 8, 17, 6, 15, 55),
 datetime.datetime(2015, 8, 17, 12, 14),
 datetime.datetime(2015, 8, 17, 14, 20, 5),
 datetime.datetime(2015, 8, 17, 20, 29, 6)]

In [61]:
mytabla_C41427 = create_travel_table("Barra", arrive_hour_to_initialpoint_C41427, initial_departure_hours_C41427, "Central", arrive_hour_to_finalpoint_C41427, final_departure_hours_C41427, 'C41427', '316' )
mytabla_C41427

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,316,C41427,Central_to_Barra,2015-08-17 06:36:54,2015-08-17 09:30:57,2:54
1,316,C41427,Barra_to_Central,2015-08-17 09:48:50,2015-08-17 12:14:00,2:25
2,316,C41427,Central_to_Barra,2015-08-17 14:41:06,2015-08-17 17:08:51,2:27
3,316,C41427,Barra_to_Central,2015-08-17 17:19:21,2015-08-17 20:29:06,3:09


In [76]:
#hacer lo mismo para el omnibus 'C41423'
df_20150817_line_316_C41423 = df_20150817_line_316[(df_20150817_line_316['bus_id'] == 'C41423')]
sorted_near_initial_points_C41423 = near_points_to_origin ('C41423', inverted_initial_point)
sorted_near_final_points_C41423 = near_points_to_origin ('C41423', inverted_final_point)


In [86]:
#graficando los initial y final points
map_316_C41423 = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316 = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

map_316_C41423.line(locations316)

#insertar los popups
#Pintando Circulos de radio de 155m alrededor de los puntos incial y final 
map_316_C41423.simple_marker(final_point, popup='Centro')
map_316_C41423.simple_marker(initial_point, popup='Barra')

#Punto inicial
map_316_C41423.circle_marker(location=final_point, radius=155,line_color='brown',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
map_316_C41423.circle_marker(location=initial_point, radius=155,line_color='black',
                 fill_color='#3186cc', fill_opacity=0.2)

#esto es para graficar los near points de la parada inicial y final
for row in df_20150817_line_316_C41423.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41423.polygon_marker( latlon, fill_color='white', num_sides=4, radius=6)    

for row in sorted_near_initial_points_C41423.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41423.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)
    
for row in sorted_near_final_points_C41423.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41423.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)    

embed_map(map_316_C41423)


In [77]:
initial_points_timestamp_C41423 = sorted_near_initial_points_C41423['timestamp']
final_points_timestamp_C41423 = sorted_near_final_points_C41423['timestamp']


In [84]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
initial_departure_hours_C41423 = bus_departure_hour(initial_points_timestamp_C41423)
final_departure_hours_C41423 = bus_departure_hour(final_points_timestamp_C41423)
arrive_hour_to_initialpoint_C41423 = bus_arrive_hour(initial_points_timestamp_C41423)
arrive_hour_to_finalpoint_C41423 = bus_arrive_hour(final_points_timestamp_C41423)
mytabla_C41423 = create_travel_table("Barra", arrive_hour_to_initialpoint_C41423, initial_departure_hours_C41423, "Central", arrive_hour_to_finalpoint_C41423, final_departure_hours_C41423, 'C41423', '316' )
mytabla_C41423

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,316,C41423,Barra_to_Central,2015-08-17 06:56:03,2015-08-17 09:45:04,2:49
1,316,C41423,Central_to_Barra,2015-08-17 10:04:20,2015-08-17 12:28:12,2:23
2,316,C41423,Barra_to_Central,2015-08-17 12:35:12,2015-08-17 14:53:20,2:18
3,316,C41423,Central_to_Barra,2015-08-17 15:09:04,2015-08-17 18:09:15,3:00
4,316,C41423,Barra_to_Central,2015-08-17 18:26:55,2015-08-17 20:40:04,2:13


In [85]:
#hacer lo mismo para el omnibus 'C41429'
df_20150817_line_316_C41429 = df_20150817_line_316[(df_20150817_line_316['bus_id'] == 'C41429')]
sorted_near_initial_points_C41429 = near_points_to_origin ('C41429', inverted_initial_point)
sorted_near_final_points_C41429 = near_points_to_origin ('C41429', inverted_final_point)

In [87]:
#graficando los initial y final points del bus_id C41429
map_316_C41429 = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316 = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

map_316_C41429.line(locations316)

#insertar los popups
#Pintando Circulos de radio de 155m alrededor de los puntos incial y final 
map_316_C41429.simple_marker(final_point, popup='Centro')
map_316_C41429.simple_marker(initial_point, popup='Barra')

#Punto inicial
map_316_C41429.circle_marker(location=final_point, radius=155,line_color='brown',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
map_316_C41429.circle_marker(location=initial_point, radius=155,line_color='black',
                 fill_color='#3186cc', fill_opacity=0.2)

#esto es para graficar los near points de la parada inicial y final
for row in df_20150817_line_316_C41429.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41429.polygon_marker( latlon, fill_color='white', num_sides=4, radius=6)    

for row in sorted_near_initial_points_C41429.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41429.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)
    
for row in sorted_near_final_points_C41429.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41429.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)    

embed_map(map_316_C41429)

In [92]:
initial_points_timestamp_C41429 = sorted_near_initial_points_C41429['timestamp']
final_points_timestamp_C41429 = sorted_near_final_points_C41429['timestamp']

In [93]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)

initial_departure_hours_C41429 = bus_departure_hour(initial_points_timestamp_C41429)

final_departure_hours_C41429 = bus_departure_hour(final_points_timestamp_C41429)

arrive_hour_to_initialpoint_C41429 = bus_arrive_hour(initial_points_timestamp_C41429)

arrive_hour_to_finalpoint_C41429 = bus_arrive_hour(final_points_timestamp_C41429)

mytabla_C41429 = create_travel_table("Barra", arrive_hour_to_initialpoint_C41429, initial_departure_hours_C41429, "Central", arrive_hour_to_finalpoint_C41429, final_departure_hours_C41429, 'C41429', '316' )

mytabla_C41429

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,316,C41429,Barra_to_Central,2015-08-17 07:14:13,2015-08-17 09:56:53,2:42
1,316,C41429,Central_to_Barra,2015-08-17 10:17:53,2015-08-17 12:29:57,2:12
2,316,C41429,Barra_to_Central,2015-08-17 12:52:15,2015-08-17 15:14:13,2:21
3,316,C41429,Central_to_Barra,2015-08-17 15:39:10,2015-08-17 19:02:09,3:22
4,316,C41429,Barra_to_Central,2015-08-17 19:15:51,2015-08-17 21:24:59,2:09


In [28]:
#hacer lo mismo para el omnibus 'C41426'
df_20150817_line_316_C41426 = df_20150817_line_316[(df_20150817_line_316['bus_id'] == 'C41426')]
sorted_near_initial_points_C41426 = near_points_to_origin ('C41426', inverted_initial_point)
sorted_near_final_points_C41426 = near_points_to_origin ('C41426', inverted_final_point)

In [29]:
#graficando los initial y final points del bus_id C41426
map_316_C41426 = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316 = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

map_316_C41426.line(locations316)

#insertar los popups
#Pintando Circulos de radio de 155m alrededor de los puntos incial y final 
map_316_C41426.simple_marker(final_point, popup='Centro')
map_316_C41426.simple_marker(initial_point, popup='Barra')

#Punto inicial
map_316_C41426.circle_marker(location=final_point, radius=155,line_color='brown',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
map_316_C41426.circle_marker(location=initial_point, radius=155,line_color='black',
                 fill_color='#3186cc', fill_opacity=0.2)

#esto es para graficar los near points de la parada inicial y final
for row in df_20150817_line_316_C41426.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41426.polygon_marker( latlon, fill_color='white', num_sides=4, radius=6)    

for row in sorted_near_initial_points_C41426.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41426.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)
    
for row in sorted_near_final_points_C41426.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41426.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)    

embed_map(map_316_C41426)

In [30]:
initial_points_timestamp_C41426 = sorted_near_initial_points_C41426['timestamp']
final_points_timestamp_C41426 = sorted_near_final_points_C41426['timestamp']


In [48]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)

initial_departure_hours_C41426 = bus_departure_hour(initial_points_timestamp_C41426)

final_departure_hours_C41426 = bus_departure_hour(final_points_timestamp_C41426)

arrive_hour_to_initialpoint_C41426 = bus_arrive_hour(initial_points_timestamp_C41426)

arrive_hour_to_finalpoint_C41426 = bus_arrive_hour(final_points_timestamp_C41426)

mytabla_C41426 = create_travel_table("Barra", arrive_hour_to_initialpoint_C41426, initial_departure_hours_C41426, "Central", arrive_hour_to_finalpoint_C41426, final_departure_hours_C41426, 'C41426', '316' )

mytabla_C41426

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time


In [53]:
#hacer lo mismo para el omnibus 'C41421'
df_20150817_line_316_C41421 = df_20150817_line_316[(df_20150817_line_316['bus_id'] == 'C41421')]
sorted_near_initial_points_C41421 = near_points_to_origin ('C41421', inverted_initial_point)
sorted_near_final_points_C41421 = near_points_to_origin ('C41421', inverted_final_point)

In [54]:
#graficando los initial y final points del bus_id C41426
map_316_C41421 = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316 = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

map_316_C41421.line(locations316)

#insertar los popups
#Pintando Circulos de radio de 155m alrededor de los puntos incial y final 
map_316_C41421.simple_marker(final_point, popup='Centro')
map_316_C41421.simple_marker(initial_point, popup='Barra')

#Punto inicial
map_316_C41421.circle_marker(location=final_point, radius=155,line_color='brown',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
map_316_C41421.circle_marker(location=initial_point, radius=155,line_color='black',
                 fill_color='#3186cc', fill_opacity=0.2)

#esto es para graficar los near points de la parada inicial y final
for row in df_20150817_line_316_C41421.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41421.polygon_marker( latlon, fill_color='white', num_sides=4, radius=6)    

for row in sorted_near_initial_points_C41421.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41421.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)
    
for row in sorted_near_final_points_C41421.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41421.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)    

embed_map(map_316_C41421)

In [88]:
#graficando los puntos de las primeras observaciones del GPS, que muestran que el omnibus en su primer viaje p
#para la barra sale desde el garaje sin pasar por el punto de la Central
map_316_C41421_0 = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations316 = []
for row in onibus316_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations316.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

map_316_C41421_0.line(locations316)

lost_points = df_20150817_line_316_C41421[(df_20150817_line_316_C41421['timestamp'] < '17-08-2015 09:18:09' )]


#insertar los popups
#Pintando Circulos de radio de 155m alrededor de los puntos incial y final 
map_316_C41421_0.simple_marker(final_point, popup='Centro')
map_316_C41421_0.simple_marker(initial_point, popup='Barra')

#Punto inicial
map_316_C41421_0.circle_marker(location=final_point, radius=155,line_color='brown',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
map_316_C41421_0.circle_marker(location=initial_point, radius=155,line_color='black',
                 fill_color='#3186cc', fill_opacity=0.2)


for row in lost_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    map_316_C41421_0.polygon_marker( latlon, fill_color='orange', num_sides=4, radius=6)  
    
latlong = [-22.857813, -43.245251 ]
map_316_C41421_0.polygon_marker( latlong, fill_color='black', num_sides=4, radius=6)  

embed_map(map_316_C41421_0)


In [79]:
lost_points

,timestamp,bus_id,line,latitude,longitude,speed
676487,17-08-2015 06:19:47,C41421,316,-22.857813,-43.245251,0.74
677510,17-08-2015 06:20:57,C41421,316,-22.857965,-43.245514,1.11
680843,17-08-2015 06:22:08,C41421,316,-22.857796,-43.245262,0.74
681990,17-08-2015 06:22:43,C41421,316,-22.857950,-43.245007,0.93
762853,17-08-2015 06:49:49,C41421,316,-22.857866,-43.244705,0.19
766197,17-08-2015 06:50:59,C41421,316,-22.857891,-43.244705,0.00
769510,17-08-2015 06:52:08,C41421,316,-22.857576,-43.245094,2.96
770544,17-08-2015 06:52:44,C41421,316,-22.857437,-43.245029,0.00
859300,17-08-2015 07:20:24,C41421,316,-22.858067,-43.245213,0.00
863104,17-08-2015 07:22:45,C41421,316,-22.858072,-43.245190,0.00


In [78]:
len(lost_points)

21

In [62]:
initial_points_timestamp_C41421 = sorted_near_initial_points_C41421['timestamp']
final_points_timestamp_C41421 = sorted_near_final_points_C41421['timestamp']
final_points_timestamp_C41421

0    17-08-2015 12:21:09
1    17-08-2015 12:22:19
2    17-08-2015 12:22:54
3    17-08-2015 17:52:50
4    17-08-2015 17:53:25
5    17-08-2015 22:23:20
6    17-08-2015 22:23:21
7    17-08-2015 22:23:30
Name: timestamp, dtype: object

In [67]:
#Llamada a la funcion bus_departure_hour  (devuelve todas las horas en que salio el omnibus del punto unicial y final)

initial_departure_hours_C41421 = bus_departure_hour(initial_points_timestamp_C41421)

final_departure_hours_C41421 = bus_departure_hour(final_points_timestamp_C41421)

arrive_hour_to_initialpoint_C41421 = bus_arrive_hour(initial_points_timestamp_C41421)

arrive_hour_to_finalpoint_C41421 = bus_arrive_hour(final_points_timestamp_C41421)

mytabla_C41421 = create_travel_table("Barra", arrive_hour_to_initialpoint_C41421, initial_departure_hours_C41421, "Central", arrive_hour_to_finalpoint_C41421, final_departure_hours_C41421, 'C41421', '316' )

mytabla_C41421


,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,316,C41421,Barra_to_Central,2015-08-17 09:22:48,2015-08-17 12:21:09,2:58
1,316,C41421,Central_to_Barra,2015-08-17 12:22:54,2015-08-17 14:53:04,2:30
2,316,C41421,Barra_to_Central,2015-08-17 14:53:04,2015-08-17 17:52:50,2:59
3,316,C41421,Central_to_Barra,2015-08-17 17:53:25,2015-08-17 20:52:21,2:58
4,316,C41421,Barra_to_Central,2015-08-17 20:53:31,2015-08-17 22:23:20,1:29


In [69]:
#Funcion que crea la tabla de tiempo de viaje para cada bus_id de los que transitaron el el dia
def all_travel_table (line, punto_ini_coord_invertidas, name_punto_inicio, punto_fin_coord_invertidas, name_punto_final, array_busId):
    new_df = pd.DataFrame(columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])  
    for busId in array_busId:
        sorted_near_initial_points1 = near_points_to_origin(busId, punto_ini_coord_invertidas)
        sorted_near_final_points1 = near_points_to_origin(busId, punto_fin_coord_invertidas)
        initial_points_timestamp1 = sorted_near_initial_points1 ['timestamp']
        final_points_timestamp1 = sorted_near_final_points1 ['timestamp']
        #chequear si la lista initial_points_timestamp es diferente de vacio
        if len(initial_points_timestamp1) and len(final_points_timestamp1):
            vilaIsabel_departure_hours1 = bus_departure_hour(initial_points_timestamp1)
            first_point_departure_hour = bus_departure_hour (initial_points_timestamp1)
            last_point_departure_hour = bus_departure_hour (final_points_timestamp1)
            first_point_arrive_hour = bus_arrive_hour(initial_points_timestamp1)
            last_point_arrive_hour = bus_arrive_hour(final_points_timestamp1)
            my_time_table = create_travel_table(name_punto_inicio, first_point_arrive_hour, first_point_departure_hour, name_punto_final, last_point_arrive_hour, last_point_departure_hour, busId, line)
            new_df = new_df.append(my_time_table, ignore_index=True)
    return new_df

In [70]:
query = all_travel_table ('316', inverted_initial_point, 'Barra', inverted_final_point, 'Central', array_busId)
query

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,316,C41422,Central_to_Barra,2015-08-17 05:58:58,2015-08-17 07:59:48,2:00
1,316,C41422,Barra_to_Central,2015-08-17 08:09:09,2015-08-17 11:07:18,2:58
2,316,C41422,Central_to_Barra,2015-08-17 11:18:58,2015-08-17 13:47:21,2:28
3,316,C41422,Barra_to_Central,2015-08-17 13:52:01,2015-08-17 15:50:19,1:58
4,316,C41424,Barra_to_Central,2015-08-17 07:34:47,2015-08-17 10:16:24,2:41
5,316,C41424,Central_to_Barra,2015-08-17 10:46:10,2015-08-17 12:59:59,2:13
6,316,C41424,Barra_to_Central,2015-08-17 13:09:55,2015-08-17 15:14:46,2:04
7,316,C41424,Central_to_Barra,2015-08-17 15:32:06,2015-08-17 18:53:01,3:20
8,316,C41424,Barra_to_Central,2015-08-17 19:09:49,2015-08-17 20:59:46,1:49
9,316,C41427,Central_to_Barra,2015-08-17 06:36:54,2015-08-17 09:30:57,2:54


In [72]:
direction_central_barra = query[(query['Direction'] == 'Central_to_Barra')]
direction_central_barra_sorted = direction_central_barra.sort(["Departure_Hour"])
direction_central_barra_sorted


,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,316,C41422,Central_to_Barra,2015-08-17 05:58:58,2015-08-17 07:59:48,2:00
9,316,C41427,Central_to_Barra,2015-08-17 06:36:54,2015-08-17 09:30:57,2:54
14,316,C41423,Central_to_Barra,2015-08-17 10:04:20,2015-08-17 12:28:12,2:23
19,316,C41429,Central_to_Barra,2015-08-17 10:17:53,2015-08-17 12:29:57,2:12
5,316,C41424,Central_to_Barra,2015-08-17 10:46:10,2015-08-17 12:59:59,2:13
2,316,C41422,Central_to_Barra,2015-08-17 11:18:58,2015-08-17 13:47:21,2:28
24,316,C41421,Central_to_Barra,2015-08-17 12:22:54,2015-08-17 14:53:04,2:30
11,316,C41427,Central_to_Barra,2015-08-17 14:41:06,2015-08-17 17:08:51,2:27
16,316,C41423,Central_to_Barra,2015-08-17 15:09:04,2015-08-17 18:09:15,3:00
7,316,C41424,Central_to_Barra,2015-08-17 15:32:06,2015-08-17 18:53:01,3:20


In [73]:
direction_barra_central = query[(query['Direction'] == 'Barra_to_Central')]
direction_barra_central_sorted = direction_barra_central.sort(["Departure_Hour"])
direction_barra_central_sorted

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
13,316,C41423,Barra_to_Central,2015-08-17 06:56:03,2015-08-17 09:45:04,2:49
18,316,C41429,Barra_to_Central,2015-08-17 07:14:13,2015-08-17 09:56:53,2:42
4,316,C41424,Barra_to_Central,2015-08-17 07:34:47,2015-08-17 10:16:24,2:41
1,316,C41422,Barra_to_Central,2015-08-17 08:09:09,2015-08-17 11:07:18,2:58
23,316,C41421,Barra_to_Central,2015-08-17 09:22:48,2015-08-17 12:21:09,2:58
10,316,C41427,Barra_to_Central,2015-08-17 09:48:50,2015-08-17 12:14:00,2:25
15,316,C41423,Barra_to_Central,2015-08-17 12:35:12,2015-08-17 14:53:20,2:18
20,316,C41429,Barra_to_Central,2015-08-17 12:52:15,2015-08-17 15:14:13,2:21
6,316,C41424,Barra_to_Central,2015-08-17 13:09:55,2015-08-17 15:14:46,2:04
3,316,C41422,Barra_to_Central,2015-08-17 13:52:01,2015-08-17 15:50:19,1:58
